# 爬蟲- 自由時報評論網-內容分類-政治

內容只有到7月, 而且局限在特定專欄, 媒體, 投書

In [ ]:
# 若選內容分類的財經新聞, 可以用selenium動態拉下十頁作法

# !pip install selenium
# !apt-get update # to update ubuntu to correctly run apt install
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# import sys
# sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver.get("https://talk.ltn.com.tw/pe")

for i in range(1,10):
  driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
  time.sleep(1)

html = BeautifulSoup(driver.page_source,'html.parser')
# print(html.prettify())

for block in html.find_all('a',class_='tit'):
  if block.has_attr("href"):
    print(block.text, block['href'])


driver.close()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options



自由開講》公投是認知作戰的下一個戰場
 https://talk.ltn.com.tw/article/breakingnews/3727121

自由開講》國民黨的「街舞」？
 https://talk.ltn.com.tw/article/breakingnews/3726925

自由開講》曾見證林靜儀專業、沒架子 她當選會是台中人福氣！
 https://talk.ltn.com.tw/article/breakingnews/3726843

自由開講》國民黨正在師法中共以「亂」致勝？！
 https://talk.ltn.com.tw/article/breakingnews/3726780

自由開講》總統專機安全上路 核四試運轉至今沒過
 https://talk.ltn.com.tw/article/breakingnews/3726074

胡，怎麼說》國民黨推4公投若通過，台灣斷骨斷筋，碎碎不安！
 https://talk.ltn.com.tw/article/breakingnews/3724295

自由開講》對照各界說法 看不出核四前廠長的專業
 https://talk.ltn.com.tw/article/breakingnews/3724830

自由開講》馮世寬應該還給國軍一個真相
 https://talk.ltn.com.tw/article/breakingnews/3724447

法操》未經檢察官聲請，法院可以直接沒收第三人財產嗎？
 https://talk.ltn.com.tw/article/breakingnews/3723594

自由開講》媽祖是信徒們的，不該屬於某基金會獨有！
 https://talk.ltn.com.tw/article/breakingnews/3723380

自由開講》黨國圖騰「轉型正義」的文化問題
 https://talk.ltn.com.tw/article/breakingnews/3723346

自由開講》國際放射師節─希望政府能正視飽受過勞與低薪壓榨困境的放射師
 https://talk.ltn.com.tw/article/breakingnews/3723263

自由開講》四個公投逼迫台灣投降
 https://talk.ltn.com.tw/articl

# 自由評論網

1. 社論 "https://talk.ltn.com.tw/list/8"
2. 自由共和國 "https://talk.ltn.com.tw/list/7" (還有包含非政治的主題)
3. 新聞線上 "https://talk.ltn.com.tw/list/9"
4. 專欄 "https://talk.ltn.com.tw/column" (有很多專欄講政治相關的)
5. 媒體 "https://talk.ltn.com.tw/media" (沒有很多講政治)
6. 投書 "https://talk.ltn.com.tw/forum" (還有包含非政治的主題, 只有到十月)


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import datetime
import re

def getday(n):
  # 獲取今天以前(-)或以後(+)幾天的日期
  today = datetime.date.today()
  oneday = today + datetime.timedelta(days=n)
  return oneday

def get_news_content(url):
  res = requests.get(url)
  html = BeautifulSoup(res.text, "html.parser")
  contents = html.find("div",class_="text")

  content = contents.find_all("p")
  text = []
  for i,c in enumerate(content):
    if c.has_attr("class"):
      pass
    else:
      if i == 0:
        author = re.sub('\W', '', c.text)
      else:
        text.append(c.text)
  return author, ''.join(text)

def get_talk_list(keyword, start_time=getday(5).strftime('%Y%m%d'), end_time=datetime.date.today().strftime('%Y%m%d'), page=1):
  # default starttime是三個月
  url = f'https://search.ltn.com.tw/list?keyword={keyword}&start_time={start_time}&end_time={end_time}&sort=date&type=talk&page={page}'
  res = requests.get(url)
  html = BeautifulSoup(res.text, "html.parser")
  n_results = html.find("div", class_="mark")
  ul = html.find("ul", class_="list boxTitle")

  news={"title":[], "content":[], "author":[]}#, "tag":[]}
  for a in ul.find_all("a", class_="tit"):
    news["title"].append(a.text)
    author, content = get_news_content(a["href"])
    news["author"].append(author)
    news["content"].append(content)
  return news

news = get_talk_list("柯文哲")
# print(news)
df = pd.DataFrame(news)
df.to_csv("news.csv", encoding="utf-8", index=False)

In [ ]:
from snownlp import SnowNLP # 使用
from snownlp import seg  # 分詞庫
from snownlp import sentiment # 情感分詞
from snownlp import normal #停用詞處理